In [ ]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from crimebb import *

In [ ]:
YEAR="2019"

In [ ]:
DATA_PATH="../../data/"
SQL_PATH = f"{DATA_PATH}sql/{YEAR}/"
CSV_PATH = f"{DATA_PATH}csv/{YEAR}/"
CSV_SUMMARY = f"{CSV_PATH}summary/"

In [ ]:
verifyDir(CSV_PATH)
verifyDir(CSV_SUMMARY)

### Database connection

In [ ]:
config = openFile(f'{DATA_PATH}/db_config.json')
config_file = config['postgres_connection']
config_file

### Listing db sizes

In [ ]:
list_dbs, path_dbs, dict_dbs = get_db_names(f"{SQL_PATH}/*.sql")

print("DB Name\n", list_dbs,"\n")

In [ ]:
db_dict = getDBsSize(config_file, list_dbs)

In [ ]:
pdCSVdb(db_dict, year_studied=YEAR)

### Listing tables sizes

In [ ]:
getTableSize(config_file, list_dbs)

### List Tables

In [ ]:
tables_dict = listDBtables(config_file, list_dbs)

In [ ]:
tables_dict

### Converting to CSV

In [ ]:
for db_name in list_dbs:
    list_tables = tables_dict[db_name]
    print(list_tables)
    for cur_table in list_tables:
        if "HF" in cur_table:
            continue
        cmd_table_to_csv(f"{db_name}", f"{cur_table}", f"{CSV_PATH}{db_name}/", passwd=config_file["password"])

### Listing CSV files

In [ ]:
tables_dict = {}

for db_name in list_dbs:
    db_tables = [db_.split("/")[-1] for db_ in glob.glob(f"{CSV_PATH}/{db_name}/*.csv")]
    tables_dict[db_name] = set(db_tables.copy())
    print(db_name, db_tables)

In [ ]:
tables_dict

### Joining

In [ ]:
tables_final_list = set.intersection(*tables_dict.values())
tables_final_list

In [ ]:
for cur_table in tables_final_list:
    print(cur_table)
    tab_df = pd.DataFrame()
    for db_name in list_dbs:
        db_path = f"{CSV_PATH}{db_name}/"
        db_tables = [db_.split("\\")[-1] for db_ in glob.glob(f"{db_path}/*")]

        table_df = pd.read_csv(f"{db_path}{cur_table}", sep=',', low_memory=False)
        tab_df = pd.concat([tab_df, table_df], ignore_index=True)
    print("num lines:", len(tab_df))
    tab_df.to_csv(f"{CSV_SUMMARY}{cur_table}", sep='\t', index=False)

In [ ]:
if YEAR=="2019":
    import shutil
    
    csv_files = glob.glob(f"{CSV_SUMMARY}/*.csv")
    for csv_orig in csv_files:
        if "forum" in csv_orig:
            shutil.move(csv_orig, f"{csv_orig.split('forum')[0]}boards.csv")
        elif "votes" in csv_orig:
            shutil.move(csv_orig, f"{csv_orig.split('reputationvotes')[0]}votes.csv")
        else:
            shutil.move(csv_orig, f"{csv_orig.split('.csv')[0]}s.csv")